In [22]:
import warnings
from io import BytesIO
from pathlib import Path
from urllib.parse import urlparse
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile, is_zipfile
import os

import pandas as pd
import socceraction.vaep.features as features
import socceraction.vaep.labels as labels

from sklearn.metrics import brier_score_loss, roc_auc_score
from socceraction.spadl.wyscout import convert_to_actions
from socceraction.vaep.formula import value
import socceraction.vaep.formula as vaepformula
from tqdm.notebook import tqdm
from xgboost import XGBClassifier
import tqdm
import random

import socceraction.atomic.spadl as atomicspadl
import socceraction.atomic.vaep.features as fs
import socceraction.atomic.vaep.labels as lab

import socceraction.atomic.spadl as atomicspadl
import socceraction.atomic.vaep.formula as vaepformula

from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)


In [23]:
%load_ext autoreload
%autoreload 2
from socceraction.data.statsbomb import StatsBombLoader
import socceraction.spadl as spadl
import socceraction.atomic.spadl as atomicspadl
from socceraction.data.wyscout import PublicWyscoutLoader



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
wyscout = PublicWyscoutLoader(root = '/Users/leonardoyang/Desktop/Tesi/Wyscout/raw/data', download=False)

In [25]:
comp = [
     'England'
     #'France',
     #'Germany',
     #'Italy',
     #'Spain'
]

In [26]:
##DA qui

In [27]:
competitions = wyscout.competitions()
competitions.info()

<class 'pandera.typing.pandas.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   competition_id      7 non-null      int64 
 1   season_id           7 non-null      int64 
 2   country_name        7 non-null      object
 3   competition_name    7 non-null      object
 4   competition_gender  7 non-null      object
 5   season_name         7 non-null      object
dtypes: int64(2), object(4)
memory usage: 464.0+ bytes


In [28]:
selected_competitions = competitions[(competitions['country_name'] =='England')]
selected_competitions

,competition_id,season_id,country_name,competition_name,competition_gender,season_name
1,364,181150,England,English first division,male,2017/2018


In [182]:
teams

,team_id,team_name_short,team_name
0,1646,Burnley,Burnley FC
1,1659,AFC Bournemouth,AFC Bournemouth
0,1628,Crystal Palace,Crystal Palace FC
1,1627,West Bromwich Albion,West Bromwich Albion FC
0,1609,Arsenal,Arsenal FC
1,1673,Huddersfield Town,Huddersfield Town FC
0,1651,Brighton & Hove Albion,Brighton & Hove Albion FC
1,1612,Liverpool,Liverpool FC
0,1644,Watford,Watford FC
1,1611,Manchester United,Manchester United FC


### Append home and away goals to the game dataset

In [29]:
## Create empty list for DataFrames
dfs_matches = []

## Loop through the competition files for the selected competition(s) and append DataFrame to dfs_matches list
for competition in selected_competitions['country_name']:
    competition_name = competition.replace(' ', '_')
    file_matches = f'matches_{competition_name}.json'
    df_matches = pd.read_json('/Users/leonardoyang/Desktop/Tesi/Wyscout/raw/Data/' + file_matches)
    dfs_matches.append(df_matches)

## Concatenate DataFrames to one DataFrame
df_matches = pd.concat(dfs_matches)
df_matches= df_matches.reset_index()

In [30]:
home_score, away_score = [],[]
for i in range(len(df_matches)) : 
    home_id, away_id = df_matches['teamsData'][i].keys()
    if df_matches['teamsData'][i][home_id]['side'] == 'away':
        home_id, away_id = away_id, home_id
    home_score.append(df_matches['teamsData'][i][home_id]['score'])
    away_score.append(df_matches['teamsData'][i][away_id]['score'])


In [31]:
df_matches['home_score'] = home_score
df_matches['away_score'] = away_score
df_matches = df_matches[['seasonId','wyId','label','competitionId', 'home_score','away_score']]
df_matches.rename(columns={'seasonId':'season_id', 'wyId':'game_id','competitionId':'competition_id'}, inplace = True)

In [32]:
df_matches.head()

,season_id,game_id,label,competition_id,home_score,away_score
0,181150,2500089,"Burnley - AFC Bournemouth, 1 - 2",364,1,2
1,181150,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0",364,2,0
2,181150,2500091,"Huddersfield Town - Arsenal, 0 - 1",364,0,1
3,181150,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0",364,4,0
4,181150,2500093,"Manchester United - Watford, 1 - 0",364,1,0


In [33]:
# Get games from all selected competitions
games = pd.concat([
    wyscout.games(row.competition_id, row.season_id)
    for row in selected_competitions.itertuples()
])

In [34]:
games.head()

,game_id,competition_id,season_id,game_date,game_day,home_team_id,away_team_id
0,2500089,364,181150,2018-05-13 14:00:00,38,1646,1659
1,2500090,364,181150,2018-05-13 14:00:00,38,1628,1627
2,2500091,364,181150,2018-05-13 14:00:00,38,1673,1609
3,2500092,364,181150,2018-05-13 14:00:00,38,1612,1651
4,2500093,364,181150,2018-05-13 14:00:00,38,1611,1644


In [35]:
#Append goals to each match
match_goals = pd.merge(games, df_matches, on =['game_id','competition_id','season_id'])
match_goals.drop(['label'],axis = 1, inplace = True)
games = match_goals

In [36]:
games[["home_team_id", "away_team_id", "game_date","home_score", "away_score"]]

,home_team_id,away_team_id,game_date,home_score,away_score
0,1646,1659,2018-05-13 14:00:00,1,2
1,1628,1627,2018-05-13 14:00:00,2,0
2,1673,1609,2018-05-13 14:00:00,0,1
3,1612,1651,2018-05-13 14:00:00,4,0
4,1611,1644,2018-05-13 14:00:00,1,0
...,...,...,...,...,...
375,1623,1639,2017-08-12 14:00:00,1,0
376,1619,10531,2017-08-12 14:00:00,0,0
377,1627,1659,2017-08-12 14:00:00,1,0
378,1644,1612,2017-08-12 11:30:00,3,3


In [37]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 379
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   game_id         380 non-null    int64         
 1   competition_id  380 non-null    int64         
 2   season_id       380 non-null    int64         
 3   game_date       380 non-null    datetime64[ns]
 4   game_day        380 non-null    int64         
 5   home_team_id    380 non-null    int64         
 6   away_team_id    380 non-null    int64         
 7   home_score      380 non-null    int64         
 8   away_score      380 non-null    int64         
dtypes: datetime64[ns](1), int64(8)
memory usage: 29.7 KB


### Convert event to actions

# DO NOT RUN

In [18]:
### DO NOT RUN
#Convert events to SPADL 
games_verbose = tqdm.tqdm(list(games.itertuples()), desc="Loading game data")
teams, players = [], []
actions = {}
atomic_actions = {}

for game in games_verbose:
    # load data
    teams.append(wyscout.teams(game.game_id))
    players.append(wyscout.players(game.game_id))
    events = wyscout.events(game.game_id)
    # convert data
    actions[game.game_id] = convert_to_actions(events, game.home_team_id)
    atomic_actions[game.game_id] = atomicspadl.convert_to_atomic(actions[game.game_id])

teams = pd.concat(teams).drop_duplicates(subset="team_id")
players = pd.concat(players)

Loading game data: 100%|██████████| 380/380 [1:52:50<00:00, 17.82s/it]


In [38]:
#Store everything in the HDF
datafolder = "/Users/leonardoyang/Desktop/data-fifa/Eng"

In [19]:
# Create data folder if it doesn't exist
if not os.path.exists(datafolder):
    os.mkdir(datafolder)
    print(f"Directory {datafolder} created.")

spadl_h5 = os.path.join(datafolder, "atomic_spadl-wyscout.h5")

# Store all spadl data in h5-file
with pd.HDFStore(spadl_h5) as spadlstore:
    spadlstore["competitions"] = selected_competitions
    spadlstore["games"] = games
    spadlstore["teams"] = teams
    spadlstore["players"] = players[['player_id', 'player_name', 'nickname']].drop_duplicates(subset='player_id')
    spadlstore["player_games"] = players[['player_id', 'game_id', 'team_id', 'is_starter', 'minutes_played']]
    for game_id in actions.keys():
        spadlstore[f"actions/game_{game_id}"] = atomic_actions[game_id]

In [39]:
#To upload the DF
spadl_h5 = os.path.join(datafolder, "atomic_spadl-wyscout.h5")

In [40]:
pd.HDFStore(spadl_h5)['teams'].reset_index().drop('index', axis =1)

,team_id,team_name_short,team_name
0,1646,Burnley,Burnley FC
1,1659,AFC Bournemouth,AFC Bournemouth
2,1628,Crystal Palace,Crystal Palace FC
3,1627,West Bromwich Albion,West Bromwich Albion FC
4,1609,Arsenal,Arsenal FC
5,1673,Huddersfield Town,Huddersfield Town FC
6,1651,Brighton & Hove Albion,Brighton & Hove Albion FC
7,1612,Liverpool,Liverpool FC
8,1644,Watford,Watford FC
9,1611,Manchester United,Manchester United FC


In [43]:
#Select a game
with pd.HDFStore(spadl_h5) as spadlstore:
    games = (
        spadlstore["games"]
        .merge(spadlstore["competitions"], how='left')
        .merge(spadlstore["teams"].add_prefix('home_'), how='left')
        .merge(spadlstore["teams"].add_prefix('away_'), how='left'))
    # Select England vs Belgium game at World Cup
    game = games[(games.country_name == "England") 
                  & (games.home_team_name_short == "Brighton & Hove Albion")
                  & (games.away_team_name_short== "Liverpool")]
    game_id = game.game_id.values[0]
    atomic_actions = spadlstore[f"actions/game_{game_id}"]
    atomic_actions = (
        atomic_actions
        .merge(atomicspadl.actiontypes_df(), how="left")
        .merge(spadl.bodyparts_df(), how="left")
        .merge(spadlstore["players"], how="left")
        .merge(spadlstore["teams"], how="left")
    )

# use nickname if available else use full name
atomic_actions["player_name"] = atomic_actions[["nickname", "player_name"]].apply(lambda x: x[0] if x[0] else x[1],axis=1)
del atomic_actions['nickname']
atomic_actions[:5]

,game_id,original_event_id,action_id,period_id,time_seconds,team_id,player_id,x,y,dx,dy,type_id,bodypart_id,type_name,bodypart_name,player_name,team_name_short,team_name
0,2499861,211308308,0,1,4.107425,1612,8319,50.40,34.68,24.15,3.4,0,0,pass,foot,J. Milner,Liverpool,Liverpool FC
1,2499861,211308308,1,1,5.166622,1612,25393,74.55,38.08,0.00,0.0,23,0,receival,foot,D. Lovren,Liverpool,Liverpool FC
2,2499861,211308310,2,1,6.225819,1612,25393,74.55,38.08,-3.15,-23.8,0,0,pass,foot,D. Lovren,Liverpool,Liverpool FC
3,2499861,211308310,3,1,7.954884,1612,116,71.40,14.28,0.00,0.0,23,0,receival,foot,G. Wijnaldum,Liverpool,Liverpool FC
4,2499861,211308314,4,1,9.683950,1612,116,71.40,14.28,-4.20,23.8,0,0,pass,foot,G. Wijnaldum,Liverpool,Liverpool FC


In [237]:
#select a goal to plot
import matplotsoccer

for shot in list(atomic_actions[(atomic_actions.type_name == "goal")][2:3].index):
    a = atomic_actions[shot-7:shot+1].copy()

    a["start_x"] = a.x
    a["start_y"] = a.y
    a["end_x"] = a.x + a.dx
    a["end_y"] = a.y + a.dy

    g = game.iloc[0]
    minute = int((a.period_id.values[0] - 1) * 45 + a.time_seconds.values[0] // 60)
    game_info = f"{g.game_date}, {g.home_team_name} - {g.away_team_name} : {g.home_score}-{g.away_score} , {a[-1:].player_name.values[0]} {minute + 1}'"
    print(game_info)

    def nice_time(row):
        minute = int((row.period_id-1) * 45 + row.time_seconds // 60)
        second = int(row.time_seconds % 60)
        return f"{minute}m{second}s"

    a["nice_time"] = a.apply(nice_time,axis=1)
    labels = a[["nice_time", "type_name", "player_name", "team_name"]]

    matplotsoccer.actions(
        location=a[["start_x", "start_y", "end_x", "end_y"]],
        action_type=a.type_name,
        team= a.team_name,
        label=labels,
        labeltitle=["time", "actiontype", "player_name", "team"],
        zoom=False,
        figsize=6
    )

AttributeError: 'Pandas' object has no attribute 'iloc'

## Compute features and labels

In [112]:
features_h5 = os.path.join(datafolder, "atomic_features.h5")
labels_h5 = os.path.join(datafolder, "atomic_labels.h5")

In [115]:
games = pd.HDFStore(spadl_h5)['games']
print("nb of games:", len(games))

nb of games: 380


In [156]:
xfns = [
    fs.actiontype,
    fs.actiontype_onehot,
    fs.bodypart,
    fs.bodypart_onehot,
    fs.goalscore,
    fs.location,
    fs.movement_polar,
    fs.polar,
    fs.direction,
    fs.team,
    fs.time,
    fs.time_delta
]

with pd.HDFStore(spadl_h5) as spadlstore, pd.HDFStore(features_h5) as featurestore:
    for game in tqdm.tqdm(list(games.itertuples()),desc=f"Generating and storing features in {features_h5}"):
        actions = spadlstore[f"actions/game_{game.game_id}"]
        gamestates = fs.gamestates(atomicspadl.add_names(actions), nb_prev_actions=3) #Each gamestate is represented as the # previous actions
        gamestates = fs.play_left_to_right(gamestates, game.home_team_id)

        X = pd.concat([fn(gamestates) for fn in xfns], axis=1)
        featurestore[f"game_{game.game_id}"] = X

Generating and storing features in /Users/leonardoyang/Desktop/data-fifa/Eng/atomic_features.h5: 100%|██████████| 380/380 [01:13<00:00,  5.18it/s]


In [157]:
yfns = [lab.scores, lab.concedes, lab.goal_from_shot]

with pd.HDFStore(spadl_h5) as spadlstore, pd.HDFStore(labels_h5) as labelstore:
    for game in tqdm.tqdm(list(games.itertuples()), desc=f"Computing and storing labels in {labels_h5}"):
        actions = spadlstore[f"actions/game_{game.game_id}"]
        Y = pd.concat([fn(atomicspadl.add_names(actions)) for fn in yfns], axis=1)
        labelstore[f"game_{game.game_id}"] = Y


Computing and storing labels in /Users/leonardoyang/Desktop/data-fifa/Eng/atomic_labels.h5: 100%|██████████| 380/380 [00:54<00:00,  7.00it/s]


### Compute Probabilities

In [158]:
predictions_h5 = os.path.join(datafolder, "atomic-predictions-one-action.h5")

In [159]:
traingames, testgames = train_test_split(games, test_size=0.3, random_state=42, shuffle=False)

In [160]:
testgames

,game_id,competition_id,season_id,game_date,game_day,home_team_id,away_team_id,home_score,away_score
266,2499834,364,181150,2017-11-18 15:00:00,12,1631,1625,0,2
267,2499835,364,181150,2017-11-18 15:00:00,12,1612,1619,3,0
268,2499838,364,181150,2017-11-18 15:00:00,12,1627,1610,0,4
269,2499830,364,181150,2017-11-18 12:30:00,12,1609,1624,2,0
270,2499819,364,181150,2017-11-05 16:30:00,11,1610,1611,1,0
...,...,...,...,...,...,...,...,...,...
375,2499723,364,181150,2017-08-12 14:00:00,1,1623,1639,1,0
376,2499726,364,181150,2017-08-12 14:00:00,1,1619,10531,0,0
377,2499728,364,181150,2017-08-12 14:00:00,1,1627,1659,1,0
378,2499727,364,181150,2017-08-12 11:30:00,1,1644,1612,3,3


In [162]:
# 1. Select feature set X
xfns = [
    #fs.actiontype,
    fs.actiontype_onehot,
    #fs.bodypart,
    fs.bodypart_onehot,
    fs.goalscore,
    fs.location,
    fs.polar,
    fs.movement_polar,
    fs.direction,
    fs.team,
    fs.time,
    fs.time_delta
]
nb_prev_actions = 3

Xcols = fs.feature_column_names(xfns, nb_prev_actions)

def getXY(games, Xcols):
    # generate the columns of the selected feature
    X = []
    for game_id in tqdm.tqdm(games.game_id, desc="Selecting features"):
        Xi = pd.HDFStore(features_h5) [f"game_{game_id}"]
        X.append(Xi[Xcols])
    X = pd.concat(X).reset_index(drop=True)

    # 2. Select label Y
    Ycols = ["scores", "concedes"]
    Y = []
    for game_id in tqdm.tqdm(games.game_id, desc="Selecting label"):
        Yi = pd.HDFStore(labels_h5)[f"game_{game_id}"]
        Y.append(Yi[Ycols])
    Y = pd.concat(Y).reset_index(drop=True)
    return X, Y

X,Y = getXY(traingames, Xcols)
#print("X:", list(X.columns))
#print("Y:", list(Y.columns))

#X = X.fillna(0)

Selecting label: 100%|██████████| 266/266 [00:02<00:00, 111.25it/s]


## Train 

In [166]:
%%time
# 3. train classifiers F(X) = Y
import xgboost

Y_hat = pd.DataFrame()
models = {}
for col in list(Y.columns):
    model = xgboost.XGBClassifier(n_estimators=50, max_depth=3, n_jobs=-3, verbosity=1)
    model.fit(X, Y[col])
    models[col] = model

CPU times: user 9min 46s, sys: 4.59 s, total: 9min 51s
Wall time: 2min 43s


In [167]:
testX, testY = getXY(testgames,Xcols)

Selecting label: 100%|██████████| 114/114 [00:00<00:00, 121.66it/s]


In [168]:
from sklearn.metrics import brier_score_loss, roc_auc_score, log_loss

def evaluate(y, y_hat):
    p = sum(y) / len(y)
    base = [p] * len(y)
    brier = brier_score_loss(y, y_hat)
    print(f"  Brier score: %.5f (%.5f)" % (brier, brier / brier_score_loss(y, base)))
    ll = log_loss(y, y_hat)
    print(f"  log loss score: %.5f (%.5f)" % (ll, ll / log_loss(y, base)))
    print(f"  ROC AUC: %.5f" % roc_auc_score(y, y_hat))

for col in testY.columns:
    Y_hat[col] = [p[1] for p in models[col].predict_proba(testX)]
    print(f"### Y: {col} ###")
    evaluate(testY[col], Y_hat[col])


### Y: scores ###
  Brier score: 0.00796 (0.82839)
  log loss score: 0.03990 (0.73051)
  ROC AUC: 0.86236
### Y: concedes ###
  Brier score: 0.00151 (0.93271)
  log loss score: 0.00946 (0.78724)
  ROC AUC: 0.85340


In [169]:
# get rows with game id per action
A = []
for game_id in tqdm.tqdm(testgames.game_id, "Loading actions of each game"):
    Ai = pd.HDFStore(spadl_h5) [f"actions/game_{game_id}"]
    A.append(Ai[["game_id"]])
A = pd.concat(A)
A = A.reset_index(drop=True)

# concatenate action game id rows with predictions and save per game
grouped_predictions = pd.concat([A, Y_hat], axis=1).groupby("game_id")
for k,df in tqdm.tqdm(grouped_predictions, desc="Saving predictions per game"):
    df = df.reset_index(drop=True)
    df[Y_hat.columns].to_hdf(predictions_h5, f"game_{int(k)}")

Saving predictions per game: 100%|██████████| 114/114 [00:01<00:00, 57.35it/s]


## VAEP

In [170]:
with pd.HDFStore(spadl_h5) as spadlstore:
    games = (
        spadlstore["games"]
        .merge(spadlstore["competitions"], how='left')
        .merge(spadlstore["teams"].add_prefix('home_'), how='left')
        .merge(spadlstore["teams"].add_prefix('away_'), how='left'))
    players = spadlstore["players"]
    teams = spadlstore["teams"]
print("nb of games:", len(testgames))

nb of games: 114


In [171]:
A = []
for game in tqdm.tqdm(list(testgames.itertuples()), desc="Loading actions"):
    actions = pd.HDFStore(spadl_h5) [f"actions/game_{game.game_id}"]
    actions = (
        atomicspadl.add_names(actions)
        .merge(players, how="left")
        .merge(teams, how="left",)
        .sort_values(["game_id", "period_id", "action_id"])
        .reset_index(drop=True)
    )
    preds = pd.HDFStore(predictions_h5) [f"game_{game.game_id}"]
    values = vaepformula.value(actions, preds.scores, preds.concedes)
    A.append(pd.concat([actions, preds, values], axis=1))
A = pd.concat(A).sort_values(["game_id", "period_id", "time_seconds"]).reset_index(drop=True)
A.columns

Loading actions: 100%|██████████| 114/114 [00:06<00:00, 18.23it/s]


Index(['game_id', 'original_event_id', 'action_id', 'period_id',
       'time_seconds', 'team_id', 'player_id', 'x', 'y', 'dx', 'dy', 'type_id',
       'bodypart_id', 'type_name', 'bodypart_name', 'player_name', 'nickname',
       'team_name_short', 'team_name', 'scores', 'concedes', 'offensive_value',
       'defensive_value', 'vaep_value'],
      dtype='object')

In [231]:
#Df with players and roles
df_players = pd.read_json('/Users/leonardoyang/Desktop/Tesi/Wyscout/raw/players.json')
df_players = df_players[['wyId','shortName','role']]
Role = []
for i in range(len(df_players)):
    Role.append(df_players['role'][i]['name'])

df_players['Role'] = Role
df_players.drop('role', axis = 1,inplace = True)

In [236]:
A["count"] = 1

# Compute each player's number of actions and total VAEP values
playersR = (
    A[["player_id", "vaep_value", "offensive_value", "defensive_value", "count"]]
    .groupby(["player_id"])
    .sum()
    .reset_index()
)
# Add player names
playersR = playersR.merge(players[["player_id", "nickname", "player_name"]], how="left")
playersR = pd.merge(playersR, df_players, left_on ='player_id', right_on = 'wyId', how = 'left').drop(['wyId','shortName'], axis = 1)
playersR["player_name"] = playersR[["nickname", "player_name"]].apply(lambda x: x[0] if x[0] else x[1],axis=1)
# Show results
playersR = playersR[["player_id", "player_name","Role", "vaep_value", "offensive_value", "defensive_value", "count"]]
playersR.sort_values("vaep_value", ascending=False)[:20]

,player_id,player_name,Role,vaep_value,offensive_value,defensive_value,count
165,8717,H. Kane,Forward,13.371799,13.105909,0.265889,681
338,120353,Mohamed Salah,Forward,12.762300,12.544324,0.217976,963
65,7905,R. Lukaku,Forward,10.578724,10.403507,0.175217,607
26,3324,Álvaro Morata,Forward,9.984560,9.825378,0.159182,607
398,340386,Gabriel Jesus,Forward,9.590411,9.519415,0.070996,621
128,8325,S. Agüero,Forward,8.500384,8.327993,0.172392,505
257,25413,A. Lacazette,Forward,7.890574,7.811057,0.079517,619
166,8726,A. Begović,Goalkeeper,7.784892,0.301426,7.483465,562
145,8480,K. Schmeichel,Goalkeeper,7.504029,0.341153,7.162876,668
210,11066,R. Sterling,Forward,7.435459,7.271693,0.163766,830


In [233]:
# Normalize for minutes played
pg = pd.HDFStore(spadl_h5) ["player_games"]
pg = pg[pg.game_id.isin(games.game_id)]
mp = pg[["player_id", "minutes_played"]].groupby("player_id").sum().reset_index()

stats = playersR.merge(mp)
stats = stats[stats.minutes_played > 1500] # at least two full games played
stats["vaep_rating"] = stats.vaep_value * 90 / stats.minutes_played
stats["offensive_rating"] = stats.offensive_value * 90 / stats.minutes_played
stats["defensive_rating"] = stats.defensive_value * 90 / stats.minutes_played
stats.sort_values("vaep_rating",ascending=False)[:10]

,player_id,player_name,Role,vaep_value,offensive_value,defensive_value,count,minutes_played,vaep_rating,offensive_rating,defensive_rating
397,340386,Gabriel Jesus,Forward,9.590411,9.519415,0.070996,621,1774,0.486548,0.482947,0.003602
25,3324,Álvaro Morata,Forward,9.984560,9.825378,0.159182,607,2180,0.412207,0.405635,0.006572
337,120353,Mohamed Salah,Forward,12.762300,12.544324,0.217976,963,3065,0.374749,0.368349,0.006401
164,8717,H. Kane,Forward,13.371799,13.105909,0.265889,681,3257,0.369500,0.362153,0.007347
127,8325,S. Agüero,Forward,8.500384,8.327993,0.172392,505,2096,0.364997,0.357595,0.007402
120,8301,J. Hart,Goalkeeper,6.729772,0.374860,6.354912,623,1848,0.327749,0.018256,0.309492
87,7974,R. Elliot,Goalkeeper,5.500489,0.439039,5.061450,625,1535,0.322504,0.025742,0.296763
55,7870,A. Ramsey,Midfielder,6.917671,6.661664,0.256008,1304,1962,0.317324,0.305581,0.011743
64,7905,R. Lukaku,Forward,10.578724,10.403507,0.175217,607,3055,0.311648,0.306486,0.005162
256,25413,A. Lacazette,Forward,7.890574,7.811057,0.079517,619,2331,0.304655,0.301585,0.003070


In [234]:
stats.sort_values("defensive_rating",ascending=False)[:25]

,player_id,player_name,Role,vaep_value,offensive_value,defensive_value,count,minutes_played,vaep_rating,offensive_rating,defensive_rating
120,8301,J. Hart,Goalkeeper,6.729772,0.374860,6.354912,623,1848,0.327749,0.018256,0.309492
87,7974,R. Elliot,Goalkeeper,5.500489,0.439039,5.061450,625,1535,0.322504,0.025742,0.296763
296,61941,F. Forster,Goalkeeper,6.058719,0.250746,5.807972,538,1924,0.283412,0.011729,0.271683
144,8480,K. Schmeichel,Goalkeeper,7.504029,0.341153,7.162876,668,3167,0.213250,0.009695,0.203555
165,8726,A. Begović,Goalkeeper,7.784892,0.301426,7.483465,562,3684,0.190185,0.007364,0.182821
149,8514,S. Mignolet,Goalkeeper,3.172587,-0.150933,3.323520,622,1825,0.156456,-0.007443,0.163900
206,10131,J. Pickford,Goalkeeper,6.816277,0.425486,6.390791,764,3635,0.168766,0.010535,0.158231
49,7847,Ł. Fabiański,Goalkeeper,6.579967,0.325374,6.254592,567,3634,0.162960,0.008058,0.154902
139,8425,W. Hennessey,Goalkeeper,4.676857,0.335126,4.341730,399,2595,0.162203,0.011623,0.150580
97,8094,J. Butland,Goalkeeper,5.949893,0.515920,5.433974,655,3369,0.158946,0.013782,0.145164


In [235]:
playersR[playersR["player_name"] == "J. Vertonghen"]

,player_id,player_name,Role,vaep_value,offensive_value,defensive_value,count
3,48,J. Vertonghen,Defender,3.214756,1.75975,1.455006,1996
